In [2]:
import os
import sys
import csv
import json
import moment
import pymysql
import datetime

import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd

## This script can generate the results data file required for plots and tables

Step 1. Set which outcome variable you want to generate data for.

In [3]:
# data_file_name = 'data/pandas_df_v2_2020-04-25.pkl'
# data_file_name = 'data/pandas_df_v3_2020-04-25.pkl'
# data_file_name = 'data/pandas_df_v3-1_2020-04-25.pkl'
data_file_name = 'data/pandas_df_v4_2020-04-25.pkl'

df = pd.read_pickle(data_file_name)
_version, date_retrieved = data_file_name.split('_df_')[1].split('.')[0].split('_')
print(_version, date_retrieved)

v4 2020-04-25


In [4]:
def propci(p, n):
    s = np.sqrt((p*(1-p)/n))
    return (p, p-1.96*s, p+1.96*s)

In [5]:
# NaN indicate people who were not covid positive
covidpos = df.dropna()

# remove patients who were intubated before they were diagnosed
print(sum(covidpos["days_to_intubation"] < 0))
covidpos = covidpos[covidpos["days_to_intubation"] >= 0]

# remove patients that died before they were diagnosed
print(sum(covidpos["days_to_death"] < 0))
covidpos = covidpos[covidpos["days_to_death"] >= 0]

# remove patients who were intubated or died more than 90 days later
print(sum(covidpos["days_to_intubation"] < 0))
covidpos = covidpos[covidpos["days_to_intubation"] < 90]
covidpos = covidpos[covidpos["days_to_death"] < 90]

covidpos['age_over_65'] = (covidpos['age'] > 65)+1-1

covidpos.describe()

101
1
0


,pat_mrn_id,intubated,days_to_intubation,died,days_to_death,age,sex,smoker,macula,compl_def,...,race_asian,race_white,race_other,race_declined,eth_hispanic,eth_nonhispanic,eth_declinedother,hx_data,co_data,age_over_65
count,2.817000e+03,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,...,2817.000000,2817.00000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000,2817.000000
mean,1.074289e+09,0.102591,19.028399,0.115726,19.886049,58.078726,0.484558,0.256656,0.021654,0.001420,...,0.018814,0.28967,0.279020,0.215832,0.430955,0.329428,0.239617,0.778133,0.853035,0.384452
std,1.249702e+08,0.303478,11.558180,0.319953,11.336636,18.619639,0.499850,0.436865,0.145578,0.037662,...,0.135893,0.45369,0.448597,0.411472,0.495298,0.470089,0.426925,0.415576,0.354134,0.486552
min,1.000011e+09,0.000000,0.000000,0.000000,0.000000,1.253936,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.003834e+09,0.000000,10.000000,0.000000,11.000000,44.298426,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
50%,1.007399e+09,0.000000,19.000000,0.000000,19.000000,59.438741,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,1.101816e+09,0.000000,28.000000,0.000000,28.000000,71.912389,1.000000,1.000000,0.000000,0.000000,...,0.000000,1.00000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,1.400038e+09,1.000000,82.000000,1.000000,82.000000,102.373717,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
covidpos[covidpos['intubated']==1].describe()

,pat_mrn_id,intubated,days_to_intubation,died,days_to_death,age,sex,smoker,macula,compl_def,...,race_asian,race_white,race_other,race_declined,eth_hispanic,eth_nonhispanic,eth_declinedother,hx_data,co_data,age_over_65
count,2.890000e+02,289.0,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.0,...,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.00000
mean,1.085816e+09,1.0,3.543253,0.391003,20.986159,62.493254,0.598616,0.276817,0.038062,0.0,...,0.010381,0.245675,0.314879,0.221453,0.487889,0.276817,0.235294,0.761246,0.923875,0.49481
std,1.423143e+08,0.0,3.836823,0.488822,11.967541,15.264491,0.491029,0.448201,0.191679,0.0,...,0.101531,0.431233,0.465273,0.415945,0.500720,0.448201,0.424918,0.427062,0.265657,0.50084
min,1.000051e+09,1.0,0.000000,0.000000,0.000000,12.605065,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.003798e+09,1.0,1.000000,0.000000,12.000000,54.513347,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000
50%,1.006726e+09,1.0,2.000000,0.000000,20.000000,64.911704,1.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000
75%,1.102090e+09,1.0,5.000000,1.000000,30.000000,72.643395,1.000000,1.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.00000
max,1.400037e+09,1.0,28.000000,1.000000,80.000000,94.132786,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [7]:
propci(0.347193, 481)

(0.347193, 0.30464673754986016, 0.3897392624501398)

In [8]:
covidpos[covidpos['died']==1].describe()

,pat_mrn_id,intubated,days_to_intubation,died,days_to_death,age,sex,smoker,macula,compl_def,...,race_asian,race_white,race_other,race_declined,eth_hispanic,eth_nonhispanic,eth_declinedother,hx_data,co_data,age_over_65
count,3.260000e+02,326.000000,326.000000,326.0,326.000000,326.000000,326.000000,326.000000,326.000000,326.0,...,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000,326.000000
mean,1.058528e+09,0.346626,14.690184,1.0,10.432515,75.656803,0.567485,0.337423,0.049080,0.0,...,0.009202,0.285276,0.294479,0.220859,0.481595,0.285276,0.233129,0.855828,0.880368,0.806748
std,1.162655e+08,0.476627,10.891875,0.0,9.394272,13.414496,0.496186,0.473558,0.216367,0.0,...,0.095634,0.452240,0.456509,0.415463,0.500429,0.452240,0.423474,0.351804,0.325029,0.395456
min,1.000051e+09,0.000000,0.000000,1.0,0.000000,12.605065,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.003503e+09,0.000000,4.000000,1.0,4.000000,68.608487,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
50%,1.005169e+09,0.000000,14.500000,1.0,8.000000,77.577002,1.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
75%,1.010823e+09,1.000000,24.000000,1.0,13.000000,85.265572,1.000000,1.000000,0.000000,0.0,...,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,1.400037e+09,1.000000,55.000000,1.0,49.000000,102.373717,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
